In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from pathlib import Path
import json 

# Resnet from scratch 
from tensorflow.keras.layers import Conv2D, Input, MaxPool2D, BatchNormalization, ReLU, Flatten, ZeroPadding2D, Add, GlobalAveragePooling2D

2024-12-28 14:58:21.059905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735397901.173573    1888 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735397901.206284    1888 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-28 14:58:21.482976: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Pretrained ResNet-50 - 1

In [7]:
base_pretrained_resnet50 = ResNet50(
    include_top = False,
    weights = "imagenet",
    input_shape = (224,224,3),
    pooling = "avg"
)

outputs = Dense(512, activation = "relu")(base_pretrained_resnet50.output) 
outputs = Dense(5, activation = "softmax")(outputs) 

model_resnet50_pretrained_1 =  Model(
    inputs = base_pretrained_resnet50.input,
    outputs = outputs
)

model_resnet50_pretrained_1.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 24,586,245 (93.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

#### Resnet18 - 1 
* Written from scratch

In [43]:
# Lovely top of the resnet-architecture
def resnet18_1(): 
    
    filters = { 
        1 : 64, 
        2 : 128,
        3 : 256, 
        4 : 512
    }
    
    def residual_block(block_type, x, identity = True): 
        shortcut = x
        first_stride = 1
        first_padding = "same"
        if identity == False:
            first_stride = 2
            first_padding = "valid"
            shortcut = Conv2D(kernel_size=1, filters=filters[block_type], strides=2)(shortcut) 
            shortcut = BatchNormalization()(shortcut)
            
        x = Conv2D(kernel_size=3, filters=filters[block_type],strides=first_stride, padding = first_padding)(x)
        if identity == False: 
            x = ZeroPadding2D(padding=((0,1),(1,0)))(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(kernel_size=3, filters=filters[block_type], strides=1, padding="same")(x)
        x = BatchNormalization()(x)
        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x
        
    inputs = Input(shape=(224,224,3)) 
    x = ZeroPadding2D(padding=3)(inputs)
    x = Conv2D(kernel_size=7, filters = 64, strides=2)(x) 
    x = BatchNormalization()(x)
    x = ReLU()(x) 
    x = ZeroPadding2D(padding=1)(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)
    
    x = residual_block(1,x)
    x = residual_block(1,x)
    
    x = residual_block(2,x,False)
    x = residual_block(2,x)
    
    x = residual_block(3,x,False)
    x = residual_block(3,x)
    
    x = residual_block(4,x,False)
    x = residual_block(4,x)

    x = GlobalAveragePooling2D()(x)

    x = Dense(512, activation="relu")(x) 
    x = Dense(5, activation="softmax")(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
model_resnet18_1 = resnet18_1()
model_resnet18_1.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_36      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_107  │ (None, 230, 230,  │          0 │ input_layer_36[0… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_337 (Conv2D) │ (None, 112, 112,  │      9,472 │ zero_padding2d_1… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_337[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_275 (ReLU)    │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_108  │ (None, 114, 114,  │          0 │ re_lu_275[0][0]   │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_35    │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_338 (Conv2D) │ (None, 56, 56,    │     36,928 │ max_pooling2d_35… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_338[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_276 (ReLU)    │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_339 (Conv2D) │ (None, 56, 56,    │     36,928 │ re_lu_276[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_339[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_129 (Add)       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d_35… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_277 (ReLU)    │ (None, 56, 56,    │          0 │ add_129[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_340 (Conv2D) │ (None, 56, 56,    │     36,928 │ re_lu_277[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_340[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_278 (ReLU)    │ (None, 56, 56,    │          0 │ batch_normalizat

 Total params: 11,456,133 (43.70 MB)

 Trainable params: 11,446,533 (43.67 MB)

 Non-trainable params: 9,600 (37.50 KB)

#### Resnet18-2

In [4]:
# Lovely top of the resnet-architecture
def resnet18_2(): 
    
    filters = { 
        1 : 64, 
        2 : 128,
        3 : 256, 
        4 : 512
    }
    
    def residual_block(block_type, x, identity = True): 
        shortcut = x
        first_stride = 1
        first_padding = "same"
        if identity == False:
            first_stride = 2
            first_padding = "valid"
            shortcut = Conv2D(kernel_size=1, filters=filters[block_type], strides=2)(shortcut) 
            shortcut = BatchNormalization()(shortcut)
            
        x = Conv2D(kernel_size=3, filters=filters[block_type],strides=first_stride, padding = first_padding)(x)
        if identity == False: 
            x = ZeroPadding2D(padding=((0,1),(1,0)))(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(kernel_size=3, filters=filters[block_type], strides=1, padding="same")(x)
        x = BatchNormalization()(x)
        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x
        
    inputs = Input(shape=(224,224,3)) 
    x = ZeroPadding2D(padding=3)(inputs)
    x = Conv2D(kernel_size=7, filters = 64, strides=2)(x) 
    x = BatchNormalization()(x)
    x = ReLU()(x) 
    x = ZeroPadding2D(padding=1)(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)
    
    x = residual_block(1,x)
    x = residual_block(1,x)
    
    x = residual_block(2,x,False)
    x = residual_block(2,x)
    
    x = residual_block(3,x,False)
    x = residual_block(3,x)
    
    x = residual_block(4,x,False)
    x = residual_block(4,x)

    x = GlobalAveragePooling2D()(x)

    x = Dense(5, activation="softmax")(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
model_resnet18_2 = resnet18_2()
model_resnet18_2.summary()

I0000 00:00:1735397933.839903    1888 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ re_lu[0][0]       │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │     36,928 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,928 │ re_lu_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 56, 56,    │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     36,928 │ re_lu_2[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat

 Total params: 11,193,477 (42.70 MB)

 Trainable params: 11,183,877 (42.66 MB)

 Non-trainable params: 9,600 (37.50 KB)

#### Resnet50 - 1

In [3]:
def resnet50_1():
    
    filters_1 = {
        1 : 64, 
        2 : 128,
        3 : 256, 
        4 : 512
    }
    
    filters_2 = {
        1 : 256,
        2 : 512,
        3 : 1024,
        4 : 2048
    } 
    
    def residual_block(block_type, x, identity = True):
        first_stride = 1
        if not identity: 
            first_stride = 2
            
        shortcut = x
        
        x = Conv2D(kernel_size=1, filters=filters_1[block_type], strides = first_stride)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(kernel_size=3, filters=filters_1[block_type], strides = 1, padding = "same")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x) 
        x = Conv2D(kernel_size=1, filters=filters_2[block_type], strides=1)(x)
        x = BatchNormalization()(x) 
        
        if x.shape != shortcut.shape:
            shortcut = Conv2D(kernel_size=1, filters=filters_2[block_type], strides=first_stride)(shortcut)
            shortcut = BatchNormalization()(shortcut) 
        
        x = Add()([x, shortcut])
        
        return x
        
    inputs = Input(shape=(224,224,3)) 
    x = ZeroPadding2D(padding=3)(inputs)
    x = Conv2D(kernel_size=7, filters = 64, strides= 2)(x)
    x = BatchNormalization()(x) 
    x = ReLU()(x)
    x = ZeroPadding2D(padding=1)(x) 
    x = MaxPool2D(pool_size=3, strides=2)(x)
    
    x = residual_block(1,x)
    x = residual_block(1,x)
    x = residual_block(1,x)

    x = residual_block(2,x,False)
    x = residual_block(2,x)
    x = residual_block(2,x)
    x = residual_block(2,x)
    
    x = residual_block(3,x,False)
    x = residual_block(3,x)
    x = residual_block(3,x)
    x = residual_block(3,x)
    x = residual_block(3,x)
    x = residual_block(3,x)

    x = residual_block(4,x,False)
    x = residual_block(4,x)
    x = residual_block(4,x)

    x = GlobalAveragePooling2D()(x) 

    x = Dense(512, activation="relu")(x)
    x = Dense(5, activation="softmax")(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
model_resnet50_1 = resnet50_1() 
model_resnet50_1.summary()

I0000 00:00:1735382566.418005     780 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ re_lu[0][0]       │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,928 │ re_lu_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     16,640 │ re_lu_2[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     16,640 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_4[0][0]  

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 24,586,245 (93.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

#### Resnet50 - 2

In [3]:
def resnet50_2():
    
    filters_1 = {
        1 : 64, 
        2 : 128,
        3 : 256, 
        4 : 512
    }
    
    filters_2 = {
        1 : 256,
        2 : 512,
        3 : 1024,
        4 : 2048
    } 
    
    def residual_block(block_type, x, identity = True):
        first_stride = 1
        if not identity: 
            first_stride = 2
            
        shortcut = x
        
        x = Conv2D(kernel_size=1, filters=filters_1[block_type], strides = first_stride)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(kernel_size=3, filters=filters_1[block_type], strides = 1, padding = "same")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x) 
        x = Conv2D(kernel_size=1, filters=filters_2[block_type], strides=1)(x)
        x = BatchNormalization()(x) 
        
        if x.shape != shortcut.shape:
            shortcut = Conv2D(kernel_size=1, filters=filters_2[block_type], strides=first_stride)(shortcut)
            shortcut = BatchNormalization()(shortcut) 
        
        x = Add()([x, shortcut])
        
        return x
        
    inputs = Input(shape=(224,224,3)) 
    x = ZeroPadding2D(padding=3)(inputs)
    x = Conv2D(kernel_size=7, filters = 64, strides= 2)(x)
    x = BatchNormalization()(x) 
    x = ReLU()(x)
    x = ZeroPadding2D(padding=1)(x) 
    x = MaxPool2D(pool_size=3, strides=2)(x)
    
    x = residual_block(1,x)
    x = residual_block(1,x)
    x = residual_block(1,x)

    x = residual_block(2,x,False)
    x = residual_block(2,x)
    x = residual_block(2,x)
    x = residual_block(2,x)
    
    x = residual_block(3,x,False)
    x = residual_block(3,x)
    x = residual_block(3,x)
    x = residual_block(3,x)
    x = residual_block(3,x)
    x = residual_block(3,x)

    x = residual_block(4,x,False)
    x = residual_block(4,x)
    x = residual_block(4,x)

    x = GlobalAveragePooling2D()(x) 
    
    x = Dense(5, activation="softmax")(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model
    
model_resnet50_2 = resnet50_2() 
model_resnet50_2.summary()

I0000 00:00:1735363681.123632    1472 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ re_lu[0][0]       │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,928 │ re_lu_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     16,640 │ re_lu_2[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     16,640 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_4[0][0]  

 Total params: 23,597,957 (90.02 MB)

 Trainable params: 23,544,837 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

#### Save 

In [4]:
with open("../config.json","r") as file:
    config = json.load(file)
    models_path = Path(config["models_folder"])
# model_resnet50_pretrained_1.save(models_path / "resnet50_pretrained_1.keras")
# model_resnet18_1.save(models_path / "resnet18_1.keras")
model_resnet50_1.save(models_path / "resnet50_1.keras")
# model_resnet50_2.save(models_path / "resnet50_2.keras")